In [ ]:
!pip install keybert
!pip install flair

     |████████████████████████████████| 319 kB 10.0 MB/s 
     |████████████████████████████████| 48 kB 6.5 MB/s 
     |████████████████████████████████| 1.2 MB 66.3 MB/s 
     |████████████████████████████████| 788 kB 81.0 MB/s 
     |████████████████████████████████| 64 kB 3.2 MB/s 
     |████████████████████████████████| 19.7 MB 89 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 981 kB 46.2 MB/s 
     |████████████████████████████████| 62 kB 1.0 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=4b7e2410f714024276c3559a5e616250f7c74c49ccd588b9bc94e9f81bb938f4
  Stored in directory: /root/.cache/pip/wheels/ba/e0/7e/726e872a53f7358b4b96a9975b04e98113b005cd8609a63abc
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=f6521c1b39ae7029f67264e746355d1c446c010d2e6e3b7075e89bf8983d0d99
  Stored in dir

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov  9 00:12:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
bert_path = '/gdrive/MyDrive/dacon/stack_bert/bert/uncased_L-12_H-768_A-12/'
file_path = '/gdrive/MyDrive/dacon/stack_bert/transfer-learning-on-stack-exchange-tags/data/'

In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

In [ ]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

In [ ]:
def copytree(src, dst, symlinks=False, ignore = None):
  for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)
    if os.path.isdir(s):
      shutil.copytree(s, d, symlinks, ignore)
    else:
      shutil.copy2(s, d)

In [ ]:
import os
# os.makedirs('bert')
# 존재하면 error 뜸

In [ ]:
copytree(os.path.join(bert_path), 'bert')

In [ ]:
biology = pd.read_csv(file_path+'biology.csv') #['id', 'title', 'content', 'tags']
cooking = pd.read_csv(file_path+'cooking.csv')
crypto = pd.read_csv(file_path+'crypto.csv')
diy = pd.read_csv(file_path+'diy.csv')
robotics = pd.read_csv(file_path+'robotics.csv')

# data = {'biology':biology, 'cooking':cooking, 'crypto':crypto, 'diy':diy, 'robotics':robotics}
data = biology.append(cooking, ignore_index=True)
data = data.append(crypto, ignore_index=True)
data = data.append(diy, ignore_index=True)
data = data.append(robotics, ignore_index=True)
# data = pd.concat
# data = biology+cooking+crypto+diy+robotics


In [ ]:
print(crypto)

          id  ...                                               tags
0          3  ...                       block-cipher des permutation
1          7  ...  oblivious-transfer multiparty-computation func...
2          8  ...                                         sha-1 hash
3          9  ...             hash cryptanalysis preimage-resistance
4         14  ...                          encryption rsa public-key
...      ...  ...                                                ...
10427  39758  ...                     hash hmac collision-resistance
10428  39759  ...                                       hash padding
10429  39761  ...                    perfect-secrecy forward-secrecy
10430  39762  ...                                hash diffie-hellman
10431  39768  ...                  encryption aes symmetric key-wrap

[10432 rows x 4 columns]


In [ ]:
from bs4 import BeautifulSoup as bs
from itertools import chain
def preprocess(data):
    # # data = re.sub(r'^\W+|\W+$','',data)
    # data = re.sub(r'\s',' ',data)
    # data = re.sub(r'\n',' ',data)
    removeHtmlTags = lambda x: bs(x, "html.parser").text
    data['cleaned_content'] = data.apply(removeHtmlTags)
    print("Removed html tags.")

    return data['cleaned_content']

def split_tag(tags):
    tag_list=[]
    for x in range(len(tags)):
        tag_list.append(str(tags[x]).split(' '))
    tag_list = list(chain.from_iterable(tag_list))
    uni_tag= np.unique(tag_list)
    print('uni=',len(uni_tag))
    dict_uni_tag = {string : i for i,string in enumerate(uni_tag)}
    print(dict_uni_tag)
    dict_mat_uni_tags=np.zeros((len(tags),len(uni_tag)))
    for c, i in enumerate(tags):
        for k, j in enumerate(i.split(' ')):
            dict_mat_uni_tags[c][dict_uni_tag[j]]=1
    return tag_list, uni_tag, dict_mat_uni_tags

In [ ]:
tag_list, uni_tag, dict_mat_uni_tags = split_tag(data.tags)
content = preprocess(data.content)
title = preprocess(data.title)


uni= 2668
{'120-240v': 0, '240v': 1, '2nd-preimage-resistance': 2, '3d-model': 3, '3d-printing': 4, '3d-reconstruction': 5, '3d-structure': 6, '3des': 7, 'abe': 8, 'abiogenesis': 9, 'abs': 10, 'accelerometer': 11, 'access-control': 12, 'access-panel': 13, 'accessibility': 14, 'accumulators': 15, 'acid': 16, 'acidity': 17, 'acoustic': 18, 'acoustic-cryptanalysis': 19, 'acoustic-rangefinder': 20, 'acrylic': 21, 'action-potential': 22, 'activerobot': 23, 'actuator': 24, 'adaptation': 25, 'addiction': 26, 'addition': 27, 'additives': 28, 'adhesive': 29, 'adversarial-model': 30, 'advice': 31, 'aes': 32, 'afci': 33, 'affine-cipher': 34, 'african': 35, 'aging': 36, 'agriculture': 37, 'aids': 38, 'air-conditioner': 39, 'air-conditioning': 40, 'air-filter': 41, 'air-leaks': 42, 'air-muscle': 43, 'air-quality': 44, 'alarm': 45, 'alcohol': 46, 'alcohol-content': 47, 'alfredo': 48, 'algebraic-attack': 49, 'algebraic-eraser': 50, 'algorithm': 51, 'algorithm-design': 52, 'alkalinity': 53, 'alkyd': 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Removed html tags.
Removed html tags.


In [ ]:
print(dict_mat_uni_tags)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# all_data = np.array([content, title])
all_data = content + title

# # p=list(chain.from_iterable(all_data))
# # print(content[0])
# print(all_data[0])
# print('88888888888888888')
# print(title[0])
# print('=----------')
# print(np.array(p)[0])
# print((content + title)[0])
# # for i in range(len(dict_mat_uni_tags)):
# #     all_data[2][i] = dict_mat_uni_tags[i]
# # print(all_data[2])
# print(len(data))
# # data['x'] = all_data
# # data['y'] = dict_mat_uni_tags

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 2)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([all_data[0]])
all_candidates = count.get_feature_names()

In [ ]:
print(all_candidates[:10])

['7b', '7b away', 'away', 'away observable', 'away start', 'bases', 'bases away', 'binding', 'binding site', 'codon']


In [ ]:
from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings

roberta = TransformerDocumentEmbeddings('roberta-base')
kw_model = KeyBERT(model=roberta)
keywords =kw_model.extract_keywords(all_data[1], keyphrase_ngram_range=(1, 1), stop_words='english', 
                                    use_maxsum=True, nr_candidates=20, top_n=5, diversity=0.7)
# keywords = kw_model.extract_keywords(all_data[0])
keywords2 =kw_model.extract_keywords(all_data[1], keyphrase_ngram_range=(1, 2), stop_words='english', 
                              use_maxsum=True, nr_candidates=20, top_n=15, diversity=0.7)
keywords3 =kw_model.extract_keywords(all_data[1], keyphrase_ngram_range=(2, 2), stop_words='english', 
                              use_maxsum=True, nr_candidates=20, top_n=15, diversity=0.7)
keywords4 =kw_model.extract_keywords(all_data[1], keyphrase_ngram_range=(2, 1), stop_words='english', 
                              use_maxsum=True, nr_candidates=20, top_n=5, diversity=0.7)
print(keywords)
print(keywords2)
print(keywords3)
print(keywords4)

[('tips', 0.9973), ('filtered', 0.9975), ('water', 0.9973), ('tend', 0.9976), ('degradation', 0.9966)]
[('free water', 0.9973), ('use depc', 0.9975), ('rna', 0.9977), ('prevented', 0.9978), ('filtered pipette', 0.9973), ('prevent rnase', 0.9976), ('does suggestions', 0.997), ('experiments prevented', 0.9976), ('treated rnase', 0.9975), ('regardless use', 0.9974), ('treated', 0.9971), ('water filtered', 0.9966), ('contamination rna', 0.9968), ('contamination working', 0.9972), ('rnase contamination', 0.9972)]
[('rnase free', 0.9973), ('pipette tips', 0.9977), ('issues degradation', 0.9978), ('degradation regardless', 0.9973), ('based experiments', 0.997), ('rna based', 0.9975), ('suggestions prevent', 0.9975), ('use depc', 0.9972), ('filtered pipette', 0.9971), ('does suggestions', 0.9975), ('experiments prevented', 0.9973), ('treated rnase', 0.9973), ('water filtered', 0.9971), ('contamination working', 0.997), ('rnase contamination', 0.9968)]
[]


In [ ]:
print(keywords2[0][0].split(' '))

['rnase', 'free']


In [ ]:
print(data.tags[1])

ribosome binding-sites translation synthetic-biology


In [ ]:
print(all_data[10000])

I used BLAST to find the best aligned DNA sequence to my query sequence.  The BLAST result shows a number of sequences that it calculated to be the best aligned sequences to the query sequence that I give.  Each of those aligned sequences have an e-value score representing the best hit by chance.  
My question is how can I determine if the query sequence is highly conserved with the aligned sequence, or not?  Can I determine that by saying the sequence with best hit e-value (lower e-value) is the one that is highly conserved? or are there other concepts that I need to consider in this case?
I appreciate your help!
How to Determine the Most Conserved Sequence from BLAST Result?
